In [29]:
#These lines set up inline plotting, and apply a standard size
%matplotlib inline
import matplotlib

import numpy as np
import matplotlib.pyplot as plt
import datetime
import re
from elasticsearch import Elasticsearch, exceptions as es_exceptions
from elasticsearch.helpers import scan
matplotlib.rc('font', **{'size': 15})

#Create instance of ElasticSearch using the Atlas host server
es = Elasticsearch(hosts=[{'host':'atlas-kibana.mwt2.org', 'port':9200}],timeout=60)



In [30]:
query = {
  "size": 500,
  "sort": [
    {
      "@timestamp": {
        "order": "desc",
        "unmapped_type": "boolean"
      }
    }
  ],
  "query": {
    "filtered": {
      "query": {
        "query_string": {
          "query": "payload.src-rse:CERN* AND payload.dst-rse:BNL* AND event_type:transfer-done",
          "analyze_wildcard": True,
          "lowercase_expanded_terms": False
        }
      },
      "filter": {
        "bool": {
          "must": [
            {
              "range": {
                "@timestamp": {
                  "gte": 1469454172245,
                  "lte": 1469455072245,
                  "format": "epoch_millis"
                }
              }
            }
          ],
          "must_not": []
        }
      }
    }
  },

  "script_fields": {},
  "fielddata_fields": [
    "@timestamp","duration"
  ]
}

In [61]:
response = es.search(index="rucio-events*", body=query, request_timeout=600)

fileSize = []
activity = []
startedAt = []
submittedAt = []
transferredAt = []
duration = []

for r in response['hits']['hits']:
    fileSize.append(r['_source']['payload']['file-size'])
    activity.append(r['_source']['payload']['activity'])
    startedAt.append(r['_source']['payload']['started_at'])
    submittedAt.append(r['_source']['payload']['submitted_at'])
    transferredAt.append(r['_source']['payload']['transferred_at'])
    duration.append(r['_source']['payload']['duration'])

In [67]:
from datetime import datetime
import time
def convertDateToEpoch(dates):
    epochDates = []
    for i in range(len(dates)):
        epoch = time.mktime(datetime.strptime(dates[i], "%Y-%m-%d %H:%M:%S"))
        epochDates.append(epoch)
    return epochDates


transferred = convertDateToEpoch(transferredAt)
started = convertDateToEpoch(startedAt)

durationCalc = np.subtract(transferred,started)/1000.

TypeError: Tuple or struct_time argument required

In [60]:
submitted

[1469472867.0,
 1469472870.0,
 1469472873.0,
 1469472873.0,
 1469472872.0,
 1469472873.0,
 1469472870.0,
 1469472870.0,
 1469472869.0,
 1469472872.0,
 1469472870.0,
 1469472872.0,
 1469472873.0,
 1469472873.0,
 1469472873.0,
 1469472867.0,
 1469472873.0,
 1469472871.0,
 1469472870.0,
 1469472872.0,
 1469472872.0,
 1469472871.0,
 1469472870.0,
 1469472872.0,
 1469472872.0,
 1469472872.0,
 1469472871.0,
 1469472869.0,
 1469472869.0,
 1469472872.0,
 1469472872.0,
 1469472871.0,
 1469472873.0,
 1469472867.0,
 1469472870.0,
 1469472873.0,
 1469472869.0,
 1469472870.0,
 1469472870.0,
 1469472873.0,
 1469472873.0,
 1469472872.0,
 1469472872.0,
 1469472873.0,
 1469472872.0,
 1469472870.0,
 1469472872.0,
 1469472872.0,
 1469472873.0,
 1469472873.0,
 1469472869.0,
 1469472871.0,
 1469472872.0,
 1469472872.0,
 1469472872.0,
 1469472870.0,
 1469472872.0,
 1469472873.0,
 1469472872.0,
 1469472873.0,
 1469472870.0,
 1469472872.0,
 1469472873.0,
 1469472873.0,
 1469472871.0,
 1469472870.0,
 146947287

1469472870.0
